# Welcome!

This example shows a slightly more complicated training paradigm in MultiBench. Namely, we'll run a MFAS, or MultiModal Fusion Architecture Search, system on the AVMNIST system.

This tutorial assumes you've followed along with the first tutorial, as we'll focus on the differences between this task and standard supervised learning in MultiBench.

To begin, let's clone the repo and setup our interpreter to run commands inside the folder.

In [1]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench

fatal: destination path 'MultiBench' already exists and is not an empty directory.
/content/MultiBench


Try to download the data file for MOSI using the below command. If this does not work for you, please download the data file locally, and upload it to the folder "/content/MultiBench/"

In [8]:
!mkdir data
!mkdir temp
!pip install gdown && gdown https://drive.google.com/u/0/uc?id=1KvKynJJca5tDtI5Mmp6CoRh9pQywH8Xp&export=download

mkdir: cannot create directory ‘data’: File exists
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1KvKynJJca5tDtI5Mmp6CoRh9pQywH8Xp 



In [3]:
!tar -xvzf avmnist.tar.gz 

avmnist/
avmnist/test_labels.npy
avmnist/image/
avmnist/image/train_data.npy
avmnist/image/test_data.npy
avmnist/audio/
avmnist/audio/train_data.npy
avmnist/audio/test_data.npy
avmnist/train_labels.npy
avmnist/avmnist_temp/
avmnist/avmnist_temp/train_labels.npy
avmnist/avmnist_temp/image/
avmnist/avmnist_temp/image/test_data.npy
avmnist/avmnist_temp/image/train_data.npy
avmnist/avmnist_temp/test_labels.npy


As Colab famously has bad handling of Conda env files, we'll install the dependencies manually so that it works. Please note that other systems might require installation of a long list of other dependencies.

In [4]:
!pip install memory-profiler

From here, let's import some of MultiBench and get working. First, we'll import what is required from all MultiBench programs:

In [5]:
import sys
import os
import torch
from torch import nn

Then, we'll import the AVMNIST dataloaders, and create the training, validation, and test dataloaders respectively:

In [6]:
import utils.surrogate as surr # This imports a learned cost model from configurations to accuracies.
from datasets.avmnist.get_data import get_dataloader # This imports the AVMNIST dataloader



traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/avmnist', batch_size=32)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


To train the MFAS model, you don't need to feed in a fusion layer nor a classification head, as both of those are looked after through MFAS. Instead, you just need to provide the pretrained encoder files for each modality encoder, and the associated hyperparameters:

In [ ]:
from training_structures.architecture_search import train # This imports the MFAS training method.

s_data = train(['pretrained/avmnist/image_encoder.pt', 
                'pretrained/avmnist/audio_encoder.pt'], 
               16, # Size of encoder output
               10, # Number of classes
               [(6, 12, 24), (6, 12, 24, 48, 96)], # Output of each layer within the unimodal encoders
               traindata, # Training data loader.
               validdata, # Validation data loader
               surr.SimpleRecurrentSurrogate().cuda(), # Surrogate instance
               (3, 5, 2), # Search space of the fusion layer
               epochs=6 # Number of epochs
               )


Search iteration 0 
Progressive step 0 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Acc: 0.2884
dev Acc: 0.3344
train Acc: 0.3467
dev Acc: 0.3826
train Acc: 0.3621
dev Acc: 0.3854
train Acc: 0.3738
dev Acc: 0.4144
train Acc: 0.3995
dev Acc: 0.4416
train Acc: 0.4165
dev Acc: 0.4476
train Acc: 0.1811
dev Acc: 0.2056
train Acc: 0.2689
dev Acc: 0.3054
train Acc: 0.3103
dev Acc: 0.3222
train Acc: 0.3262
dev Acc: 0.3454
train Acc: 0.3383
dev Acc: 0.3638
train Acc: 0.3448
dev Acc: 0.3634
